# Visualize recharge maps 
using geemap and Google Earth images

In [7]:
import geemap
import pandas as pd
import numpy as np
import ee
ee.Initialize()

# center over Africa
Map = geemap.Map(center=[0,40], zoom=2)

## Maps at 0.5 degree

### LMM based map

In [8]:
# add BGS recharge to the map
bgs_recharge_vis_params = {
    'min': 0,'max': 300, 
    'bands':'bgs_recharge', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_bgs_recharge = ee.Image('users/pazolka/BGS_Africa_recharge_05').rename('bgs_recharge')
Map.addLayer(ee_bgs_recharge, bgs_recharge_vis_params, 'BGS-LMM recharge 0.5', shown=False)
# fix for clean colorbar
bgs_recharge_vis_params['bands'] = 'Recharge (mmpa)'
Map.add_colorbar(bgs_recharge_vis_params)

# # BGS recharge
#Map.add_raster('Low_res_data_05/BGS_Africa_recharge_05.tif', colormap='RdYlGn', bands=[1], layer_name='BGS recharge 0.5')

### Predictor maps

In [9]:
# import LTA precipitation raster
#Map.add_raster('Recharge_files/Africa_bgs_LTA_AnnPrecip.tif', colormap='Spectral', bands=[1], layer_name='BGS Precip')

# import PET raster
#Map.add_raster('Recharge_files/Africa_bgs_LTA_AnnPET.tif', colormap='RdYlBu', bands=[1], layer_name='BGS PET')

# import aridity raster
#Map.add_raster('Recharge_files/Africa_bgs_aridity_index.tif', colormap='RdBu', bands=[1], layer_name='BGS Aridity')

# import NDVI raster
#ap.add_raster('Recharge_files/Africa_NDVI.tif', colormap='RdYlGn', bands=[1], layer_name='BGS NDVI')

# import resampled soil moisture (0-10cm) raster
#Map.add_raster('Low_res_data_05/LTA_FLDAS_Soil_moisture_Afr_05.tif', colormap='BrBG', bands=[1], layer_name='Soil moisture')

# import resampled LC raster
#Map.add_raster('Low_res_data_05/landcover_Afr_05.tif', colormap='BrBG', bands=[1], layer_name='Land cover')

### Sample points

In [11]:
# add all training sample points to the layer
# run RF_map_05.ipynb first to generate summary file
file = pd.read_csv('summary_05_z.csv')
train_points = []

arr = file.to_numpy()
    
def createGeoPoints(x):
    lon = x[1]
    lat = x[2]
    res = x[3] - x[4]
    train_points.append(ee.Feature(ee.Geometry.Point(lon,lat), 
                             {'id': x[0], 
                              'rf_residual': res,
                              'rf_residual_rel': res/x[4]*100, 
                              'obs': x[3],
                              'rf': x[4],
                             }))
    return x

np.apply_along_axis(createGeoPoints, axis=1, arr=arr)

train_pc = ee.FeatureCollection(train_points);
Map.add_ee_layer(train_pc, {'color': 'lightgreen'}, 'Train sample points', shown=False)

In [12]:
# add train sample points with high negative residual to the layer
train_pc_lres = ee.FeatureCollection(train_pc.filter(ee.Filter.eq('obs', 0.1)))
Map.add_ee_layer(train_pc_lres, {'color': '#000000'}, 'Zero-recharge points', shown=False)

# # add train sample points with high residual to the layer
# train_pc_hres = ee.FeatureCollection(train_pc.filter(ee.Filter.gt('rf_residual', 100)))
# Map.add_ee_layer(train_pc_hres, {'color': '#FF0000'}, 'Train sample points: high residual', shown=False)

In [13]:
# legend
legend_keys = ['Training and testing samples', 'Zero-recharge samples']
legend_colors = ['#90ee90', '#FF00FF']

Map.add_legend(legend_title="Groundwater recharge samples", legend_keys=legend_keys, legend_colors=legend_colors, position='bottomleft')

### RF-based map

In [14]:
# visualise RF results
#Map.add_raster('RF_recharge.tif', colormap='RdYlGn', bands=[1], layer_name='RF recharge')
# add GEE for seamless inspection
rf_recharge_vis_params = {
    'min': 0,'max': 300, 
    'bands':'rf_recharge', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_rf_recharge = ee.Image('users/pazolka/RF_recharge').rename('rf_recharge')
Map.addLayer(ee_rf_recharge, rf_recharge_vis_params, 'RF recharge', shown=False)

### RK RF based map (Python + R)

In [20]:
rk_rf_recharge_vis_params = {
    'min': 0,'max': 300, 
    'bands':'rf_recharge', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_rk_rf_recharge = ee.Image('users/pazolka/RK_RF_recharge_05').rename('rf_recharge')
Map.addLayer(ee_rk_rf_recharge, rk_rf_recharge_vis_params, 'RK RF recharge', shown=False)

### RF based map, zero-recharge sites included

In [21]:
rf_recharge_vis_params = {
    'min': 0,'max': 300, 
    'bands':'rf_recharge', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_rf_recharge_z = ee.Image('users/pazolka/RF_recharge_zero').rename('rf_recharge')
Map.addLayer(ee_rf_recharge_z, rf_recharge_vis_params, 'RF recharge zero', shown=False)

diff_recharge_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'yellow', 'blue']
}
ee_diff_zero_rech = ee_rf_recharge_z.subtract(ee_rf_recharge).rename('diff_recharge')
Map.addLayer(ee_diff_zero_rech, diff_recharge_vis_params, 'RF_z - RF', shown=False)

diff_recharge_rel_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
Map.addLayer(ee_diff_zero_rech.divide(ee_rf_recharge).multiply(100), diff_recharge_rel_vis_params, 'RF_z - RF/RF (rel %)', shown=False)

### Difference RF - LMM

In [22]:
diff_recharge_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'yellow', 'blue']
}
ee_diff_recharge = ee_rf_recharge.subtract(ee_bgs_recharge).rename('diff_recharge')
Map.addLayer(ee_diff_recharge, diff_recharge_vis_params, 'RF - LMM', shown=False)
# fix for clean colorbar
diff_recharge_vis_params['bands'] = 'abs difference (mmpa)'
Map.add_colorbar(diff_recharge_vis_params, label="RF - LMM")

diff_recharge_rel_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge_rel', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_diff_recharge_rel = ee_diff_recharge.divide(ee_bgs_recharge).multiply(100).rename('diff_recharge_rel')
Map.addLayer(ee_diff_recharge_rel, diff_recharge_rel_vis_params, 'RF - LMM /LMM (rel %)', shown=False)
# fix for clean colorbar
diff_recharge_rel_vis_params['bands'] = 'rel difference (%)'
Map.add_colorbar(diff_recharge_rel_vis_params, label="(RF-LMM)/LMM * 100%")

### Difference RK RK - LMM

In [23]:
diff_recharge_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'yellow', 'blue']
}
ee_diff_recharge = ee_rk_rf_recharge.subtract(ee_bgs_recharge).rename('diff_recharge')
Map.addLayer(ee_diff_recharge, diff_recharge_vis_params, 'RK RF - LMM', shown=False)

diff_recharge_rel_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge_rel', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_diff_recharge_rel = ee_diff_recharge.divide(ee_bgs_recharge).multiply(100).rename('diff_recharge_rel')
Map.addLayer(ee_diff_recharge_rel, diff_recharge_rel_vis_params, 'RK RF - LMM /LMM (rel %)', shown=False)

### Difference RK RF - RF

In [24]:
diff_recharge_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'yellow', 'blue']
}
ee_diff_recharge = ee_rk_rf_recharge.subtract(ee_rf_recharge).rename('diff_recharge')
Map.addLayer(ee_diff_recharge, diff_recharge_vis_params, 'RK RF - RF', shown=False)

diff_recharge_rel_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge_rel', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_diff_recharge_rel = ee_diff_recharge.divide(ee_rf_recharge).multiply(100).rename('diff_recharge_rel')
Map.addLayer(ee_diff_recharge_rel, diff_recharge_rel_vis_params, 'RK RF - RF/RF (rel %)', shown=False)

## Maps at 0.1 degree

### RF based map at 0.1 deg

In [25]:
ee_rf_recharge_01 = ee.Image('users/pazolka/RF_recharge_01').rename('rf_recharge')
rf_recharge_vis_params = {
    'min': 0,'max': 300, 
    'bands':'rf_recharge', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
Map.addLayer(ee_rf_recharge_01, rf_recharge_vis_params, 'RF recharge 01', shown=False)

### LMM based map at 0.1 deg

In [26]:
lmm_01_vis_params = {
    'min': 0,'max': 300, 
        'bands':'lmm_01', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_lmm_01_recharge = ee.Image('users/pazolka/LMM_R_01').rename('lmm_01')
Map.addLayer(ee_lmm_01_recharge, lmm_01_vis_params, 'LMM recharge 0.1', shown=False)

### RK RF based map at 0.1 deg

In [27]:
rk_rf_recharge_vis_params = {
    'min': 0,'max': 300, 
    'bands':'rf_recharge', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_rk_rf_recharge_01 = ee.Image('users/pazolka/RK_RF_recharge_01').rename('rf_recharge')
Map.addLayer(ee_rk_rf_recharge_01, rk_rf_recharge_vis_params, 'RK RF recharge 01', shown=False)

### Difference RF - LMM

In [28]:
diff_recharge_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'yellow', 'blue']
}
ee_diff_recharge = ee_rf_recharge_01.subtract(ee_lmm_01_recharge).rename('diff_recharge')
Map.addLayer(ee_diff_recharge, diff_recharge_vis_params, 'RF - LMM 0.1', shown=False)

diff_recharge_rel_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge_rel', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_diff_recharge_rel = ee_diff_recharge.divide(ee_lmm_01_recharge).multiply(100).rename('diff_recharge_rel')
Map.addLayer(ee_diff_recharge_rel, diff_recharge_rel_vis_params, 'RF - LMM/LMM 0.1 (%)', shown=False)

### Difference RK RF - LMM

In [29]:
diff_recharge_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'yellow', 'blue']
}
ee_diff_recharge = ee_rk_rf_recharge_01.subtract(ee_lmm_01_recharge).rename('diff_recharge')
Map.addLayer(ee_diff_recharge, diff_recharge_vis_params, 'RK RF - LMM 01', shown=False)

diff_recharge_rel_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge_rel', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_diff_recharge_rel = ee_diff_recharge.divide(ee_lmm_01_recharge).multiply(100).rename('diff_recharge_rel')
Map.addLayer(ee_diff_recharge_rel, diff_recharge_rel_vis_params, 'RK RF - LMM /LMM 0.1 (rel %)', shown=False)

### Difference RK RF - RF

In [30]:
diff_recharge_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge', 'palette': ['red', 'yellow', 'blue']
}
ee_diff_recharge = ee_rk_rf_recharge_01.subtract(ee_rf_recharge_01).rename('diff_recharge')
Map.addLayer(ee_diff_recharge, diff_recharge_vis_params, 'RK RF - RF 01', shown=False)

diff_recharge_rel_vis_params = {
    'min': -100,'max': 100, 
    'bands':'diff_recharge_rel', 'palette': ['red', 'pink', 'yellow', 'lightgreen', 'green', 'lightblue', 'blue']
}
ee_diff_recharge_rel = ee_diff_recharge.divide(ee_rf_recharge_01).multiply(100).rename('diff_recharge_rel')
Map.addLayer(ee_diff_recharge_rel, diff_recharge_rel_vis_params, 'RK RF - RF/RF 01 (%)', shown=False)

In [31]:
Map

Map(bottom=812.0, center=[0, 40], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [18]:
## add continent outline for better presentation
Map.add_shapefile("CRU_precip_CGIAR_AI_data/Africa_continent_shape.shp", 'outline', style={"weight": 2, 'fill':False}, fill_colors=['white'], info_mode='on_hover')